In [ ]:
import pandas as pd

df =  pd.read_csv(
    "../data/raw/mortgage_data/historical_data_2022Q1/historical_data_2022Q1.txt",
    sep="|"
)



In [ ]:
df.head()

In [ ]:
df_test = pd.read_csv("../data/raw/hdma/hmda_2017.csv")

In [ ]:
df_test.shape